<img src="img/cabecera.png?raw=1">

## Embeddings

Primero vamos a ver cómo funciona un word embedding de una forma simulada al tiempo que vemos como emplear la capa de embeddings de Keras, luego veremos como hacer sentence embedding utilizando un modulo o modelo preentrenado de Tensorflow.

In [ ]:
import numpy as np
import tensorflow as tf

### Word Embeddings

Un word embedding transforma las palabras de un texto en un vector de n dimensiones. Veamos como hacerlo con una capa de embeddings, sin entrenar y así podrás ver como instanciarla.

In [ ]:
categorias_ejemplo = ["Me","llamo","Iñigo","Montoya","soy","tú","mataste","a","mi","padre"] # Una frase o un vocabulario de ejemplo
pre_conversion = tf.keras.layers.StringLookup() # IMPORTANTE: Hay que convertir nuestro vocabulario a indices 
pre_conversion.adapt(categorias_ejemplo) # Y como ya habíamos visto, hay que hacer un fit/adapt

lookup_y_embedding = tf.keras.Sequential([tf.keras.layers.InputLayer(shape=[], dtype=tf.string), 
                                          pre_conversion,
                                          tf.keras.layers.Embedding(input_dim = pre_conversion.vocabulary_size(),
                                                                    output_dim = 2)])
# input_dim -> Tamaño del vocabulario a convertir en vectores de output_dim dimensiones

Este "modelo" no resuelve ningún tipo de problema solo pasa las palabras a traves de la capa de codificación y luego de la embeddings y genera por cadda palabra un vector de 2 dimensiones (output_dim). Pero además como no está entrenada funcionará porque tiene pesos inicializados de forma aleatoria. Es decir que si le pasamos como entrada la variable con la frase de ejemplo...

In [ ]:
pre_conversion(categorias_ejemplo)

In [ ]:
lookup_y_embedding(np.array(categorias_ejemplo))

Nos convierte cada palabra en un embedding sin sentido. 

Otra forma de hacerlo:

In [ ]:
frase = "Me llamo Iñigo Montoya"
lookup_y_embedding(np.array(frase.split()))

Para poder darle valor tendríamos que incluir nuestras dos capas (la codificadora y la de embedding) en un modelo con un objetivo determinado y la capa de embeddings se entrenaría para generar los embeddings que mejor se adapten al problema a solucionar con ese modelo

### Sentences embedding

Vamos a convertir una serie de frases en embeddings. En concreto de 50 dimensiones. Lo haremos utilizando un modelo preentrenado el nnlem-en-dim50 de Google. Internamente es un modelo word embeddings que convierte cada palabra en un embedding de 50 dimensiones y luego calcular el centroide de todos los vectores obtenidos para una frase.

In [ ]:
import tensorflow_hub as hub

hub_layer = hub.KerasLayer("https://tfhub.dev/google/nnlm-en-dim50/2") # Así se obtiene el modelo
sentence_embeddings = hub_layer(tf.constant(["To be", "Not to be"]))
sentence_embeddings.numpy().round(2)


Probemos ahora algunas cosas como por ejemplo obtener la similitud entre sentencias

In [ ]:
sentences = ['El Real Madrid lo tiene difícil para ganar al Manchester City.',
 'El Barcelona puede clasificar frente al PSG, si se esfuerza.',
 'Las tropas rusas han tomado Dubrovnik.',
 'El ejercito ucraniano se ha replegado']

In [ ]:
sentence_embeddings = hub_layer(tf.constant(sentences))

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from itertools import combinations

for (frase1,vec1),(frase2,vec2) in combinations(zip(sentences,sentence_embeddings.numpy()), r = 2):
    print(frase1,"vs",frase2, cosine_similarity([vec1],[vec2]), np.linalg.norm(vec1-vec2))

Con el coseno tendríamos algún problema con la distancia quedan mejor emparejadas. 

In [ ]:
question  = "¿Contra quién juega el Barcelona?"
pregunta = hub_layer(tf.constant([question]))
vec_q = pregunta.numpy()
distancias = []
respuestas = []

for answer,vec_a in zip(sentences,sentence_embeddings.numpy()):
    respuestas.append(answer)
    distancias.append(np.linalg.norm(vec_q-vec_a))
    
print(f"P:{question}")
print(f"R:{respuestas[np.argmin(distancias)]}")

In [ ]:
question  = "¿Qué hacen los ucranianos?"
pregunta = hub_layer(tf.constant([question]))
vec_q = pregunta.numpy()
distancias = []
respuestas = []

for answer,vec_a in zip(sentences,sentence_embeddings.numpy()):
    respuestas.append(answer)
    distancias.append(np.linalg.norm(vec_q-vec_a))
    
print(f"P:{question}")
print(f"R:{respuestas[np.argmin(distancias)]}")

In [ ]:
question  = "¿Qué ha pasado en Dubrovnik?"
pregunta = hub_layer(tf.constant([question]))
vec_q = pregunta.numpy()
distancias = []
respuestas = []

for answer,vec_a in zip(sentences,sentence_embeddings.numpy()):
    respuestas.append(answer)
    distancias.append(np.linalg.norm(vec_q-vec_a))

print(f"P:{question}")
print(f"R:{respuestas[np.argmin(distancias)]}")